In [85]:
#Importing all necessary libraries
import tkinter as tk
import random as r
import math as m

In [86]:
def readStart(filename):
    data = open(filename)
    start = data.readline()
    data.close()
    coordinates = start.split("\n")[0].split(",")
    return (int(coordinates[0]), int(coordinates[1]))

def readGoal(filename):
    data = open(filename)
    data.readline()
    goal = data.readline()
    data.close()
    coordinates = goal.split("\n")[0].split(",")
    return (int(coordinates[0]), int(coordinates[1]))

def createGrid(filename, rows):
    data = open(filename)
    for x in range(10):
        data.readline()
    grid = []
    for x in range(rows):
        grid.append(list(data.readline().split("\n")[0]))
    data.close()
    return grid
    

In [87]:

class mapNode:
    def __init__(self, block, highway):
        self.block = block
        self.highway = highway

def createRoughTerrain(arr, width, height):
    pairs = []
    for x in range(8):
        pairs.append([r.randint(0,width),r.randint(0,height)])
    for pair in pairs:
        for x in range(31):
            if pair[0] + 15 - x >= 0 and pair[0] + 15 - x < width:
                for y in range(31):
                    if pair[1] + 15 - y >= 0 and pair[1] + 15 - y < height:
                        if r.random() > 0.5:
                            arr[pair[0] + 15 - x][pair[1] + 15 - y].block = 1
    return pairs
    
def generateHighwayStart(width, height):
    start = [0,0]
    direction = [0,0]
    if r.random() > 0.5:
        startx = r.randint(0, width - 1)
        if r.random() > 0.5:
            direction = [0,-1]
            start = [startx, height - 1]
        else:
            direction = [0,1]
            start = [startx, 0]
    else:
        starty = r.randint(0, height - 1)
        if r.random() > 0.5:
            direction = [1,0]
            start = [0, starty]
        else:
            direction = [-1,0]
            start = [width - 1, starty]
    return [start, direction]
    
def createHighway(arr, width, height):
    highwayNodes = []
    highwayStart = generateHighwayStart(width, height)
    highwayNodes.append(highwayStart[0])
    direction = highwayStart[1]
    invalids = 0
    invalid = 0
    while invalids < 100:
        for x in range(20):
            nextx = highwayNodes[len(highwayNodes) - 1][0] + direction[0]
            nexty = highwayNodes[len(highwayNodes) - 1][1] + direction[1]
            if nextx >= 0 and nexty >= 0 and nextx < width and nexty < height:
                if [nextx, nexty] not in highwayNodes and arr[nextx][nexty].highway == 0:
                    highwayNodes.append([nextx, nexty])
                else:
                    invalids+=1
                    invalid = 1
                    break
            else:
                if len(highwayNodes) >= 100:
                    return highwayNodes
                else:
                    invalids+=1
                    invalid = 1
                    break
        if invalid == 1:
            highwayNodes = []
            highwayStart = generateHighwayStart(width, height)
            highwayNodes.append(highwayStart[0])
            direction = highwayStart[1]
            invalid = 0
        else:
            randFloat = r.random()
            if randFloat < 0.2:
                if direction[0] == 0:
                    direction = [direction[1],direction[0]]
                else:
                    direction = [-direction[1],-direction[0]]
            elif randFloat < 0.4 and randFloat > 0.2:
                if direction[0] == 0:
                    direction = [-direction[1],-direction[0]]
                else:
                    direction = [direction[1],direction[0]]
    return [0]
   
def generateHighways(arr, width, height):
    x = 0
    while x < 4:
        nodes = createHighway(arr, width, height)
        if len(nodes) >= 100:
            for node in nodes:
                arr[node[0]][node[1]].highway = 1
            x += 1
        else:
            for x in range(width):
                for y in range(height):
                    arr[x][y].highway = 0
            x = 0
    return
   
def createBlocks(arr, width, height):
    for x in range(width):
        for y in range(height):
            if r.random() < 0.2:
                if arr[x][y].highway == 0:
                    arr[x][y].block = 2
    return

def getEuclideanDistance(coordinate1, coordinate2):
    return m.sqrt((coordinate2[0] - coordinate1[0]) ** 2 + (coordinate2[1] - coordinate1[1]) ** 2)

def generateBorderCoordinates(width, height):
    if r.random() < 0.5:
        startx = r.randint(0, width - 1)
        if r.random() < 0.5:
            starty = r.randint(0, 19)
        else:
            starty = r.randint(height - 20, height - 1)
        return [startx, starty]
    else:
        starty = r.randint(0, height - 1)
        if r.random() < 0.5:
            startx = r.randint(0, 19)
        else:
            startx = r.randint(width - 20, width - 1)
        return [startx, starty]

def generateStartandFinish(arr, width, height):
    start = generateBorderCoordinates(width, height)
    while arr[start[0]][start[1]].block == 2:
        start = generateBorderCoordinates(width, height)
    return start               

def arrayGen(width, height, filename):
    newArr = [[mapNode(0,0) for x in range(height)] for y in range(width)]
    roughCenters = createRoughTerrain(newArr, width, height)
    generateHighways(newArr, width, height)
    createBlocks(newArr, width, height)
    start = generateStartandFinish(newArr, width, height)
    goal = generateStartandFinish(newArr, width, height)
    while getEuclideanDistance(start, goal) < 100:
        start = generateStartandFinish(newArr, width, height)
        goal = generateStartandFinish(newArr, width, height)
    file = open(filename, 'w')
    file.write('')
    file.close()
    file = open(filename, 'a')
    file.write(str(start[0]) + ',' + str(start[1]) + '\n')
    file.write(str(goal[0]) + ',' + str(goal[1]) + '\n')
    for coordinate in roughCenters:
        file.write(str(coordinate[0]) + ',' + str(coordinate[1]) + '\n')
    for x in range(width):
        for y in range(height):
            if newArr[x][y].block == 2:
                file.write('0')
            elif newArr[x][y].block == 1:
                if newArr[x][y].highway == 0:
                    file.write('2')
                else:
                    file.write('b')
            else:
                if newArr[x][y].highway == 0:
                    file.write('1')
                else:
                    file.write('a')
        file.write('\n')
    file.close()
    return newArr

class puzzleGrid:
    
    #A constructor for intializing a map. Accepts a master and 2D array as inputs. 
    #Produces a window displaying the values inside the input 2D Array
    def __init__(self, master, inArray):
        width = len(inArray)
        height = len(inArray[0])
        #Constructor loop iterates over all values in the input array and input their value in string form to a tkinter grid at an equivalent position
        for i in range(width):
            for j in range(height):
                tk.Label(master, text = (str(inArray[i][j].block) + ',' + str(inArray[i][j].highway)), padx = 0, pady = 0, relief = "sunken").grid(row=i, column=j)

In [88]:
#Performs a binary insertion, placing a new vertex in sorted order.
def vertexInsert(vertexList, vertex, start, end):
    m = (start + end)//2
    if m == start:
        vertexList.insert(m, vertex)
        return
    if vertexList[m].fVal < vertex.fVal:
        vertexInsert(vertexList, vertex, m, end)
    else:
        vertexInsert(vertexList, vertex, start, m)
#Automatically performs a binary insertion on a list of vertices.
def vertexPush(vertexList, vertex):
    vertexInsert(vertexList, vertex, 0, len(vertexList))
#Removes and returns the first vertex in the list.
def vertexPop(vertexList):
    return vertexList.pop(0)

In [89]:

#Dictionary containing inferred costs for each terrain type. Use to calculate cost for traveling between cells.
costOf = {"1": 1, "2": 2, "a":.25, "b":.5}

#Initializing fringe and closed list
#Might be better to initialize these separately and have this as its own method
#Pass it a map, and start node have it return a list of the path with [0] being start and [n] being the goal.

#Vertices are used to track A*'s progress across the grid. Each vertex has coordinate values for its position, a parent, and f, g, and h values which can be udpated.
class vertex:
    def __init__(self, coordinate, parent, terrain, fVal, gVal, hVal):
        self.coordinate = coordinate
        self.parent = parent
        self.terrain = terrain
        #Possible to initialize this last and set it equal to sum of gVal and hVal?
        self.fVal = fVal
        self.gVal = gVal
        self.hVal = hVal

class aStarSearcher:
    def __init__(self, gridWorld, startCoordinate, goalCoordinate):
        #Initializing search variables
        self.gridWorld = gridWorld
        self.startCoordinate = startCoordinate
        self.goalCoordinate = goalCoordinate
        
        self.fringe = []
        self.closedList = {}
        self.searchPath = []
    
    #Euclidean distance heuristic, for use while prototyping A* search
    def getEuclideanDistance(self, coordinate1, coordinate2):
        return m.sqrt((coordinate2[0] - coordinate1[0]) ** 2 + (coordinate2[1] - coordinate1[1]) ** 2)
    
    def getPath(self, current, path):
        totalPath = [current]
        while current.parent in path:
            current = path[current.parent]
            totalPath.append(current)
        return totalPath

    #Identifies the valid (nothing off grid) neighboring cells of the 8 adjacent cells to a coordinate and returns a list containing these neighbors initialized to vertices
    #Ignorant of parents and impassable terrain on purpose
    def getSuccessors(self, coordinate, parentCoordinate):
    #Utilizes 2D array based calculations to identify 8 adjacent cells. Assigns terrain values to each.
    
        #Return list of adjacent cells
        neighbors = []
    
        #Iterating over x values
        for x in range (-1, 2):
            for y in range(-1, 2):
                #Skipping the coordinate itself to avoid having 9 coordiantes
                if (x != 0 and y != 0) or (x != parentCoordinate[0] and y != parentCoordinate[1]):
                    try:
                        print('successor',self.gridWorld[coordinate[0]+x][coordinate[1]+y])
                        if (self.gridWorld[coordinate[0]+x][coordinate[1]+y] != '0'):
                            neighbors.append(vertex((coordinate[0]+x, coordinate[1]+y), None, self.gridWorld[coordinate[0]+x][coordinate[1]+y], 0, 0, 0))
                    except IndexError:
                        continue    
        return neighbors
    
    #Checks if the vertex is in the heap, priority queue fringe. Accepts a vertex class object and returns True or False
    def inFringe(self, vertex):
        for x in self.fringe:
            if vertex.coordinate == x.coordinate:
                return True
        return False
    
    #Identifies the cost of traveling between two vertices using the terrain and coordinate values
    def getCost(self, vertex1, vertex2):
        x_difference = vertex2.coordinate[0] - vertex1.coordinate[0]
        y_difference = vertex2.coordinate[1] - vertex1.coordinate[0]
        #If the x difference or y difference is 0, then the movement is horizontal/vertical
        #This means that cost is just 1/2 cost of vertex 1 + 1/2 cost vertex 2
        if x_difference == 0 or y_difference == 0:
            return 0.5 * costOf[self.gridWorld[vertex1.coordinate[0]][vertex1.coordinate[1]]] + 0.5 * costOf[self.gridWorld[vertex2.coordinate[0]][vertex2.coordinate[1]]]
        #If both differences are not 0, then this means movement is diagonal
        #Cost is 1/2 * sqrt(2) * cost of the vertices
        else:
            diagonal_1 = 0.5 * costOf[self.gridWorld[vertex1.coordinate[0]][vertex1.coordinate[1]]] * m.sqrt(2)
            diagonal_2 = 0.5 * costOf[self.gridWorld[vertex2.coordinate[0]][vertex2.coordinate[1]]] * m.sqrt(2)
            return diagonal_1 + diagonal_2
        
    def updateVertex(self, current, successor):
        #If the successor is new (infinite g value)
        if current.gVal + self.getEuclideanDistance(current.coordinate, successor.coordinate) < successor.gVal:
            print('updating successor gval')
            #Assigning f,g,h values and parent to successor
            successor.gVal = current.gVal + self.getCost(current, successor)
            successor.hVal = self.getEuclideanDistance(successor.coordinate, self.goalCoordinate)
            successor.fVal = successor.gVal + successor.hVal
            successor.parent = current.coordinate
            
            #Updates priority of successor by removing and readding the successor to the fringe. Otherwise adds new successor to fringe
            if self.inFringe(successor):
                self.fringe.remove(successor)
            vertexPush(self.fringe, successor)

    #Will be using euclidean distance heuristic written by K. Pei, should make algorithm modular as we develop the project.        
    def aStarSearch(self):
    
        #Initializing start vertex
        #Must be an unblocked cell
        startVertex = vertex(self.startCoordinate, None, 1, 0, 0, 0)
        startVertex.parent = startVertex.coordinate
        startVertex.hVal = self.getEuclideanDistance(startVertex.coordinate, self.goalCoordinate)
        startVertex.fVal = startVertex.gVal + startVertex.hVal
    
        #Adding the start vertice to the fringe
        vertexPush(self.fringe, startVertex)
    
        #Main searching loop
        while len(self.fringe)>0:
            search = vertexPop(self.fringe)
            #Checking if goal found
            if search.coordinate[0] == self.goalCoordinate[0] and search.coordinate[1] == self.goalCoordinate[1]:
                print("Path found")
                return self.getPath(search, self.searchPath)
            #Setting current node to have been visited and checked
            self.closedList[search.coordinate] = True
        
            #Identifying successors
            newSuccessors = self.getSuccessors(search.coordinate, search.parent)
            print(len(newSuccessors), 'successors left')
            #Iterating through successors
            for successor in newSuccessors:
                #Checking if successor was already visited
                if (successor.coordinate in self.closedList) == False:
                    #Checking if successor is not in the fringe, it is a new successor. Assign g and parent
                    if not self.inFringe(successor):
                        print(successor.coordinate, 'not in fringe, gval is infinite')
                        successor.gVal = m.inf
                        successor.parent = None
                    #Update the values of the fringe nodes based on the new current node (popped from fringe)
                    self.updateVertex(search, successor)
            print(len(self.fringe),"fringe nodes left")
        return None


In [90]:
filename = 'test.txt'

x = arrayGen(120, 160, filename)

startCoordinate = readStart(filename)
goalCoordinate = readGoal(filename)

print('start',startCoordinate)
print('goal',goalCoordinate)

gridWorld = createGrid(filename,120)
print(gridWorld[startCoordinate[0]][startCoordinate[1]])

pathFinder = aStarSearcher(gridWorld, startCoordinate, goalCoordinate)
output = pathFinder.aStarSearch()
print(output)


start (112, 5)
goal (8, 2)
1
successor 1
successor 1
successor 1
successor 1
successor 1
successor 0
successor 1
successor 1
successor 1
8 successors left
(111, 4) not in fringe, gval is infinite
updating successor gval
(111, 5) not in fringe, gval is infinite
updating successor gval
(111, 6) not in fringe, gval is infinite
updating successor gval
(112, 4) not in fringe, gval is infinite
updating successor gval
(113, 4) not in fringe, gval is infinite
updating successor gval
(113, 5) not in fringe, gval is infinite
updating successor gval
(113, 6) not in fringe, gval is infinite
updating successor gval
7 fringe nodes left
successor 1
successor 0
successor 0
successor 1
successor 1
successor 1
successor 1
successor 1
successor 0
6 successors left
(110, 4) not in fringe, gval is infinite
updating successor gval
7 fringe nodes left
successor 1
successor 1
successor 1
successor 1
successor 1
successor 0
successor 1
successor 1
successor 1
8 successors left
(109, 3) not in fringe, gval is i

In [91]:

class weightedAStarSearcher(aStarSearcher):
    def __init__(self, gridWorld, startCoordinate, goalCoordinate, weight, algoType):
        #Initializing search variables
        self.gridWorld = gridWorld
        self.startCoordinate = startCoordinate
        self.goalCoordinate = goalCoordinate
        if(algoType == "dijkstra"):
            self.weight = 0
        else:
            self.weight = weight
        
        self.fringe = []
        self.closedList = {}
        self.searchPath = []
    
    #Will be using euclidean distance heuristic written by K. Pei, should make algorithm modular as we develop the project.        
    def weightedSearch(self):
    
        #Initializing start vertex
        #Must be an unblocked cell
        startVertex = vertex(self.startCoordinate, self, 1, 0, 0, 0)
        startVertex.hVal = self.getEuclideanDistance(startVertex.coordinate, goalCoordinate)
        startVertex.fVal = startVertex.gVal + (self.weight * startVertex.hVal)
    
        #Adding the start vertice to the fringe
        vertexPush(self.fringe, startVertex)
    
        #Main searching loop
        while len(self.fringe)>0:
            search = vertexPop(self.fringe)
            #Checking if goal found
            if search.coordinate[0] == goalCoordinate[0] and search.coordinate[1] == goalCoordinate[1]:
                print("Path found")
                return self.getPath(search, self.searchPath)
            #Setting current node to have been visited and checked. OH WAIT ARRAYS CAN'T BE FUCKING DICTIONARY KEYS
            self.closedList[search.coordinate] = True
        
            #Identifying successors
            newSuccessors = self.getSuccessors(search.coordinate, search.parent)
        
            #Iterating through successors
            for successor in newSuccessors:
                #Checking if successor was already visited
                if self.closedList[successor.coordinate] == False:
                    #Checking if successor is not in the fringe, it is a new successor. Assign g and parent
                    if self.inFringe(successor) is not True:
                        successor.gVal = m.inf
                        successor.parent = None
                    #Update the values of the fringe nodes based on the new current node (popped from fringe)
                    self.updateVertex(search, successor)
        return None
